In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.8 MB/s 
     |████████████████████████████████| 7.6 MB 20.9 MB/s 
     |████████████████████████████████| 182 kB 63.4 MB/s 


In [3]:
!pip install datasets
!pip install contractions
!pip install difflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 6.4 MB/s 
     |████████████████████████████████| 212 kB 58.4 MB/s 
     |████████████████████████████████| 115 kB 44.4 MB/s 
     |████████████████████████████████| 127 kB 58.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 7.4 MB/s 
     |████████████████████████████████| 106 kB 56.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib


In [5]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
## for data
import datasets  #(1.13.3)
import pandas as pd  #(0.25.1)
import numpy  #(1.16.4)## for plotting
import matplotlib.pyplot as plt  #(3.1.2)
import seaborn as sns  #(0.9.0)## for preprocessing
import re
import nltk  #(3.4.5)
import contractions  #(0.0.18)## for textrank
import gensim  #(3.8.1)## for evaluation
import rouge  #(1.0.0)
import difflib## for seq2seq
from tensorflow.keras import callbacks, models, layers, preprocessing as kprocessing #(2.6.0)## for bart
import transformers  #(3.0.1)

In [7]:
dataset = datasets.load_dataset("cnn_dailymail", '3.0.0')
lst_dics = [dic for dic in dataset["train"]]## keep the first N articles if you want to keep it lite 
dtf = pd.DataFrame(lst_dics).rename(columns={"article":"text", 
      "highlights":"y"})[["text","y"]].head(20000)
dtf.head()

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

,text,y
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."


In [8]:
i = 1
print("--- Full text ---")
print(dtf["text"][i])
print("--- Summary ---")
print(dtf["y"][i])

--- Full text ---
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less li

In [9]:
dtf_train = dtf.iloc[i+1:]
dtf_test = dtf.iloc[:i+1]

In [ ]:
print(dtf_test["text"][0])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

In [10]:
def textrank(corpus, ratio=0.5):    
    if type(corpus) is str:        
       corpus = [corpus]    
    lst_summaries = [gensim.summarization.summarize(txt,  
                     ratio=ratio) for txt in corpus]    
    return lst_summaries
## Apply the function to corpus
predicted = textrank(corpus= dtf_test["text"], ratio=0.5)
predicted[i]

'Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill.\nAn inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court.\nMost often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police.\nMentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow directions, according to Leifman.\nSo, they end up on the ninth floor severely mentally disturbed, but not getting any real help because they\'re

In [15]:
def find_ex_sum(text):
  sum = gensim.summarization.summarize(txt,  ratio=0.5)


In [12]:
txt = """It's been some time since Facebook's Mark Zuckerberg went meta on us but 
if recent reports are to be considered true, the endevour of Metaverse hasn't gained 
pace that it was expected to. But now an island nation in the Pacific Ocean may start a 
new trend, not because of any pleasant reason. Tuvalu, the island country in southern Pacific Ocean, 
is planning to build a digital version of itself.Reason? Climate Change is threatening the very existence of 
Tuvalu. The country's foreign minister announced the metaverse plans in the ongoing COP27 climate summit. Foreign
 Minister Simon Kofe it was time to look at alternative solutions for his country's survival, and that included 
 Tuvalu becoming the first digitised nation in the metaverse – an online realm that uses augmented and 
 virtual reality (VR) to help users interact. Kofe gave his video address standing on a digital replica of 
 an islet threatened by rising sea levels. This is not the first time Kofe has hit the headlines. 
 Last year, in COP26 summit, he addressed those present while standing in knee-deep seawater. 
 He sought to underline threat to his nation due to rising sea levels, an effect triggered by climate change. 
 Tuvalu will be the first country to replicate itself in the metaverse, but follows both the city of Seoul and the island nation of Barbados, 
 which last year said they would enter the metaverse to provide administrative and consular services, respectively."""

sum = gensim.summarization.summarize(txt,  ratio=0.5)

In [13]:
print(sum)

But now an island nation in the Pacific Ocean may start a 
Tuvalu, the island country in southern Pacific Ocean, 
The country's foreign minister announced the metaverse plans in the ongoing COP27 climate summit.
Minister Simon Kofe it was time to look at alternative solutions for his country's survival, and that included 
Tuvalu becoming the first digitised nation in the metaverse – an online realm that uses augmented and 
Kofe gave his video address standing on a digital replica of 
This is not the first time Kofe has hit the headlines.
Last year, in COP26 summit, he addressed those present while standing in knee-deep seawater.
He sought to underline threat to his nation due to rising sea levels, an effect triggered by climate change.
Tuvalu will be the first country to replicate itself in the metaverse, but follows both the city of Seoul and the island nation of Barbados, 
